# Court Vision Pipeline
### Single Stationary Camera
1. Detect lines or corners of court in a single frame of the clip
2. Compute the homography between the image and the base world frame
3. Make image plane detections. People, Ball etc.
4. Project detections onto base world frame

In [76]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from torchvision.io import read_image
from courtvision.geometry import (
    get_corners_image,
    get_coords_world_3d_n,
    corners_world_n,
    convert_corners_to_vec,
    PadelCourt,
    compute_homography,
    project_points_to_base_plane
)
from courtvision.geometry import corners_world_3d
import numpy as np
from courtvision.vis import (
    plot_n_images_in_a_grid,
    load_timg,
    plot_3d_lines,
    plot_3d_points,
    log_court_layout
)
from courtvision.swiss import save_camera_params
import cv2
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

import rerun as rr

from courtvision.models import get_fasterrcnn_ball_detection_model
from courtvision.swiss import get_latest_file
from courtvision.trackers import Tracker




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
RUN_PLAYER_DETECTION = False
RUN_BALL_DETECTION = False
RUN_BALL_TRACKER = True

COURT_MESH_PATH = Path(
    "/Users/benjamindecharmoy/projects/courtvision/blender/basic_image.glb"
)

CALIBRATION_FILE = Path(
    "/Users/benjamindecharmoy/projects/courtvision/datasets/calibrations/v1/24_error_11.11_calibration.npz"
)
CLIP_DATA_DIR = Path(
    "/Users/benjamindecharmoy/projects/courtvision/data/frames/curated_001"
)
RAW_CLIP_PATH = Path(
    "/Users/benjamindecharmoy/projects/courtvision/data/raw/curated_001.mp4"
)
CLIP_NAME = RAW_CLIP_PATH.stem

IMAGE_TO_FLOOR_HOMOGRAPHY_FILE = Path(
    "/Users/benjamindecharmoy/projects/courtvision/data/frames/curated_001/homography.npy"
)

BALL_DETECTOR_DIR = Path(
    "/Users/benjamindecharmoy/projects/courtvision/models/ball_detector/"
)
BALL_DETECTOR_PATH = get_latest_file(BALL_DETECTOR_DIR, ".pt")
BALL_DETECTOR_MODEL_NAME = BALL_DETECTOR_PATH.stem

In [78]:
if RUN_PLAYER_DETECTION:
    from ultralytics import YOLO

    model = YOLO("yolov8n.pt")

    model.classes = [0]
    model.conf = 0.6
    model.max_det = 4
    results = model.track(
        source=RAW_CLIP_PATH.as_posix(),
        # tracker="/Users/benjamindecharmoy/projects/courtvision/bytetrack.yaml",
        tracker="/Users/benjamindecharmoy/projects/courtvision/botsort.yml",
        classes=[0],
        max_det=4,
        save=True,
    )

else:
    from pickle import load, dump

    results = load(open("results.pkl", "rb"))

In [80]:
# Load Trained Ball detection model
if RUN_BALL_DETECTION:
    ball_detector = get_fasterrcnn_ball_detection_model(BALL_DETECTOR_PATH).eval()

if RUN_BALL_TRACKER:
    tracker = Tracker(
        num_particles=1000,
        world_to_cam=torch.tensor(np.load(CALIBRATION_FILE)["camera_matrix"]),
        court_size=torch.tensor(
            [PadelCourt.width, PadelCourt.length, PadelCourt.backall_fence_height]
        ),
    )

from courtvision.trackers import StateIdx
from courtvision.geometry import (
    convert_points_to_homogeneous,
    convert_points_from_homogeneous,
)

#  def state_to_observation(state, H):
x_y_z_1_positions = tracker.states[:, : StateIdx.z + 1].rename(None)
print(x_y_z_1_positions)
convert_points_from_homogeneous((tracker.H @ x_y_z_1_positions.T).T)


# tracker.likelihood(obs_state=torch.tensor([100,100]),pred_state=tracker.states)

tensor([[50., 70.,  5.],
        [50., 70.,  5.],
        [50., 70.,  5.],
        ...,
        [50., 70.,  5.],
        [50., 70.,  5.],
        [50., 70.,  5.]])


tensor([[17064.2383, 23731.7988],
        [17064.2383, 23731.7988],
        [17064.2383, 23731.7988],
        ...,
        [17064.2383, 23731.7988],
        [17064.2383, 23731.7988],
        [17064.2383, 23731.7988]])

In [63]:
tracker.xyz

tensor([[  89.2775,   41.5601,   73.1566],
        [ 104.1247, -189.6820,   29.2836],
        [ 173.9896,   53.6179,  106.1288],
        ...,
        [  14.9235,  -94.6962,   51.7571],
        [  10.8966, -142.5561,  -13.8988],
        [  19.8083,  231.7995,   26.1076]], names=('num_particles', 'state'))

In [64]:
from collections import defaultdict
from kornia import image_to_tensor
from datetime import datetime

player_maker_radius = 5.0
base_results = defaultdict(list)
if not RUN_BALL_DETECTION:
    ball_detection_resullt = load(open("ball_detection_resullt.pkl", "rb"))
else:
    ball_detection_resullt = []
rr.init(
    f"{CLIP_NAME}-{BALL_DETECTOR_MODEL_NAME}-{datetime.now()}", spawn=True
)  # Spawn a Rerun Viewer and stream log events to it

colours_per_idx = defaultdict(lambda: (255, 255, 255))
colours_per_idx.update(
    {
        0: (0, 255, 0),
        1: (0, 0, 255),
        2: (255, 0, 0),
        3: (255, 255, 0),
        4: (255, 0, 255),
        5: (0, 255, 255),
    }
)
log_court_layout(
    camera_matrix=np.load(CALIBRATION_FILE)["camera_matrix"],
    image_width=results[0].orig_img.shape[1],
    image_height=results[0].orig_img.shape[0],
    court_mesh_path=COURT_MESH_PATH,
    translation_vector=np.load(CALIBRATION_FILE)["translation_vector"],
    rotation_vector=np.load(CALIBRATION_FILE)["rotation_vector"],
)
H = np.load(IMAGE_TO_FLOOR_HOMOGRAPHY_FILE)

for i, result in enumerate(results):
    # Start streaming results to the Rerun Viewer
    rr.set_time_sequence("play", i)
    if RUN_BALL_DETECTION:
        with torch.no_grad():
            outputs = ball_detector(
                image_to_tensor(result.orig_img, keepdim=False).float() / 255.0,
            )
        ball_detection_resullt.append(outputs)
        dump(ball_detection_resullt, open("ball_detection_resullt.pkl", "wb"))
    else:
        outputs = ball_detection_resullt[i]

    # Ball Tracker Goes here!!
    if RUN_BALL_TRACKER:
        rr.log_points(
            "world/ball_state",
            positions=tracker.xyz,
        )
    if outputs and len(outputs[0]["boxes"]) > 0:
        for bx1, by1, bx2, by2 in outputs[0]["boxes"][:4]:
            # bx1, by1, bx2, by2 = outputs[0]["boxes"][0]
            rr.log_rect(
                f"world/camera/image/Ball",
                (bx1, by1, bx2 - bx1, by2 - by1),
                color=colours_per_idx[-1],
            )
    rr.log_image("world/camera/image", result.orig_img)

    for det in result.boxes.data:
        x1, y1, x2, y2, idx, conf, cls = det
        rr.log_rect(
            f"world/camera/image/Player_{int(idx)}",
            (x1, y1, (x2 - x1), (y2 - y1)),
            color=colours_per_idx[int(idx)],
        )
        mid_feet = torch.tensor([((x1 + x2) / 2, (y2 + y2) / 2)])
        (mid_feet_base,) = project_points_to_base_plane(points=mid_feet, H=H)

        mid_feet_base_3d = (
            F.pad(mid_feet_base, (0, 1), mode="constant", value=0.0) / 10.0
        )
        # Switch Y and Z axis
        x = mid_feet_base_3d[0].item()
        y = mid_feet_base_3d[1].item()
        z = mid_feet_base_3d[2].item()
        mid_feet_base_3d[2] = player_maker_radius
        mid_feet_base_3d[1] = 200.0 - y
        mid_feet_base_3d[0] = x
        rr.log_point(
            f"world/Player_{int(idx)}",
            mid_feet_base_3d,
            radius=player_maker_radius,
            color=colours_per_idx[int(idx)],
        )

        # base_results[f"{int(idx)}_xs"].append(mid_feet_base[0].item())
        # base_results[f"{int(idx)}_ys"].append(mid_feet_base[1].item())
        # base_results[f"{int(idx)}_zs"].append(0.0)
        # base_results[f"{int(idx)}_conf"].append(conf.item())

    # if i > 10:
    #     break

    # break
# plt.imshow(image)
# results[0].boxes.boxes, results[0].boxes.data

[2023-06-06T17:38:13Z WARN  rerun::run] Failed to bind TCP address "0.0.0.0:9876". Another Rerun instance is probably running.
[2023-06-06T17:38:13Z WARN  re_renderer::importer::gltf] Textures on meshes are always sampled repeating address mode.
     exture None had ClampToEdge for s wrapping and ClampToEdge for t wrapping, these settings will be ignored


In [65]:
dd = torch.tensor([1, 2, 3])
# F.pad(dd, ( 0,1), mode="constant", value=0.0)
# swap y and z
torch.swapdims(dd, 1, 2)
dd

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
results[0].orig_img

In [ ]:
def get_court_frontwall_markings():
    outer_lines = np.array(
        [
            # Outer lines
            corners_world_3d["a_front_left"],
            corners_world_3d["b_front_right"],
            corners_world_3d["n_top_front_right"],
            corners_world_3d["m_top_front_left"],
            corners_world_3d["a_front_left"],
        ],
        dtype=np.float32,
    )
    x_offset = 3 * 100.0
    z_offset = 5 * 100.0
    play_boundary = np.array(
        [
            # Outer lines
            (
                corners_world_3d["a_front_left"][0] - x_offset,
                corners_world_3d["a_front_left"][1],
                corners_world_3d["a_front_left"][2],
            ),
            (
                corners_world_3d["b_front_right"][0] + x_offset,
                corners_world_3d["b_front_right"][1],
                corners_world_3d["b_front_right"][2],
            ),
            (
                corners_world_3d["n_top_front_right"][0] + x_offset,
                corners_world_3d["n_top_front_right"][1],
                corners_world_3d["n_top_front_right"][2] + z_offset,
            ),
            (
                corners_world_3d["m_top_front_left"][0] - x_offset,
                corners_world_3d["m_top_front_left"][1],
                corners_world_3d["m_top_front_left"][2] + z_offset,
            ),
            (
                corners_world_3d["a_front_left"][0] - x_offset,
                corners_world_3d["a_front_left"][1],
                corners_world_3d["a_front_left"][2],
            ),
        ],
        dtype=np.float32,
    )
    xs = np.array([outer_lines[:-1, 0], outer_lines[1:, 0]]).T
    xs = np.append(
        xs, np.array([play_boundary[:-1, 0], play_boundary[1:, 0]]).T, axis=0
    )

    ys = np.array([outer_lines[:-1, 1], outer_lines[1:, 1]]).T
    ys = np.append(
        ys, np.array([play_boundary[:-1, 1], play_boundary[1:, 1]]).T, axis=0
    )

    zs = np.array([outer_lines[:-1, 2], outer_lines[1:, 2]]).T
    zs = np.append(
        zs, np.array([play_boundary[:-1, 2], play_boundary[1:, 2]]).T, axis=0
    )

    return xs, ys, zs


xs, ys, zs = get_court_frontwall_markings()
plt_axis, fig = plot_3d_lines(xs=xs, ys=ys, zs=zs, view_init=(0, 90, 0))
plt_axis.set_title("")
plt_axis.set_xlabel("")
plt_axis.set_ylabel("")
plt_axis.set_xticks([])
plt_axis.set_yticks([])
plt_axis.set_zticks([])
plt_axis.spines["right"].set_visible(False)
plt_axis.spines["top"].set_visible(False)
plt_axis.spines["bottom"].set_visible(False)
plt_axis.spines["left"].set_visible(False)
plt.axis("off")
plt.axis("image")
plt.savefig("frontwall.png", bbox_inches=0)

In [ ]:
corners_world_3d

In [ ]:
def get_court_markings():
    outer_lines = np.array(
        [
            # Outer lines
            corners_world_3d["a_front_left"],
            corners_world_3d["b_front_right"],
            corners_world_3d["d_back_right"],
            corners_world_3d["c_back_left"],
            corners_world_3d["a_front_left"],
        ],
        dtype=np.float32,
    )
    inner_lines = np.array(
        [
            corners_world_3d["e_left_near_serve_line"],
            corners_world_3d["f_right_near_serve_line"],
            corners_world_3d["h_right_far_serve_line"],
            corners_world_3d["g_left_far_serve_line"],
            corners_world_3d["e_left_near_serve_line"],
        ],
        dtype=np.float32,
    )

    center_line = np.array(
        [
            corners_world_3d["k_center_line_near"],
            corners_world_3d["i_center_line_far"],
        ]
    )
    net_line = np.array(
        [
            corners_world_3d["j_net_line_left"],
            corners_world_3d["l_net_line_right"],
        ]
    )

    xs = np.array([outer_lines[:-1, 0], outer_lines[1:, 0]]).T
    xs = np.append(xs, np.array([inner_lines[:-1, 0], inner_lines[1:, 0]]).T, axis=0)
    xs = np.append(xs, np.array([center_line[:-1, 0], center_line[1:, 0]]).T, axis=0)
    xs = np.append(xs, np.array([net_line[:-1, 0], net_line[1:, 0]]).T, axis=0)

    ys = np.array([outer_lines[:-1, 1], outer_lines[1:, 1]]).T
    ys = np.append(ys, np.array([inner_lines[:-1, 1], inner_lines[1:, 1]]).T, axis=0)
    ys = np.append(ys, np.array([center_line[:-1, 1], center_line[1:, 1]]).T, axis=0)
    ys = np.append(ys, np.array([net_line[:-1, 1], net_line[1:, 1]]).T, axis=0)

    zs = np.array([outer_lines[:-1, 2], outer_lines[1:, 2]]).T
    zs = np.append(zs, np.array([inner_lines[:-1, 2], inner_lines[1:, 2]]).T, axis=0)
    zs = np.append(zs, np.array([center_line[:-1, 2], center_line[1:, 2]]).T, axis=0)
    zs = np.append(zs, np.array([net_line[:-1, 2], net_line[1:, 2]]).T, axis=0)

    return xs, ys, zs


xs, ys, zs = get_court_markings()
plt_axis, fig = plot_3d_lines(xs=xs, ys=ys, zs=zs, view_init=(90, 90, 0))
from courtvision.vis import plot_3d_points

idx = 1
# xs = np.array(base_results[f"{idx}_xs"])
# ys = np.array(base_results[f"{idx}_ys"])
# zs = np.array(base_results[f"{idx}_zs"])
plt_axis.set_title("")
plt_axis.set_xlabel("")
plt_axis.set_ylabel("")
plt_axis.set_xticks([])
plt_axis.set_yticks([])
plt_axis.set_zticks([])
plt_axis.spines["right"].set_visible(False)
plt_axis.spines["top"].set_visible(False)
plt_axis.spines["bottom"].set_visible(False)
plt_axis.spines["left"].set_visible(False)
# plot_3d_lines
plt.axis("off")
plt.axis("image")
plt_axis.margins(x=0)
plt.savefig("test.png", bbox_inches=0)

# plt_axis, _=plot_3d_points(x=xs, y=ys, z=zs, plt_axis=plt_axis ,view_init=(90, 90,0))

In [ ]:
rr.log("test", "test")

In [ ]:
from courtvision.vis import plot_3d_points, plot_3d_lines
from courtvision.geometry import get_coords_world_3d
import numpy as np

court_markings = get_coords_world_3d()
xs = np.array([court_markings[:1, 0], court_markings[1:2, 0]]).T
xs = np.append(xs, np.array([court_markings[1:2, 0], court_markings[3:4, 0]]).T, axis=0)
xs = np.append(xs, np.array([court_markings[3:4, 0], court_markings[4:5, 0]]).T, axis=0)
xs = np.append(xs, np.array([court_markings[4:5, 0], court_markings[:1, 0]]).T, axis=0)

ys = np.array([court_markings[:-1, 1], court_markings[1:, 1]]).T
zs = np.array([court_markings[:-1, 2], court_markings[1:, 2]]).T
plot_3d_lines(xs=xs, ys=ys, zs=zs)
# get_coords_world_3d()